# Accessors for WSIData

The accessor is a concept that use attributes to extend the capabilities of a class.

There are three in-built accessors in the WSIData class:

- {py:class}`get <wsidata.GetAccessor>`: Get information about the WSI
- `iter`: Iterate over the content of the WSI
- `ds`: Create deep learning datasets from the WSI

Here, we will load a WSI that have already been processed with tissue detection, tissue tiling, and feature extraction.

In your case, you can easily run these steps with `LazySlide` package.

In [2]:
from wsidata import open_wsi

URL = "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1-Small-Region.svs"
wsi = open_wsi(URL)

## Get accessor

In [4]:
wsi.get.pyramids()

,height,width,downsample
level,,,
0,2967,2220,1.0


In [ ]:
wsi.get.features_anndata("resnet50")

## Iter accessor

In [ ]:
next(wsi.iter.tissue_contours())

In [ ]:
next(wsi.iter.tissue_images())

In [ ]:
next(wsi.iter.tile_images())

## Dataset accessor

In [ ]:
dataset = wsi.ds.tile_images()

The dataset is a torch dataset that can be used to train a deep learning model. You can load it in the DataLoader and train the model.

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(dataset, batch_size=36, shuffle=True)